# Concept GNN Pipeline – GPU A100
Ce notebook exécute automatiquement toutes les étapes du projet (balayage métrique, génération, entraînement, évaluation) en partant du repo **pascalseppey/concept-gnn-pipeline**.

## 1. Pré-requis
- Runtime Colab : **GPU A100** (Runtime → Modifier le type de matériel → GPU → A100).
- Repo GitHub : `https://github.com/pascalseppey/concept-gnn-pipeline.git`.
- `requirements.txt` gère toutes les dépendances (PyTorch GPU, torch_geometric, etc.).

In [ ]:
!nvidia-smi

## 2. Cloner le repo et installer les dépendances

In [ ]:
import os
REPO_URL = 'https://github.com/pascalseppey/concept-gnn-pipeline.git'
REPO_DIR = '/content/concept-gnn-pipeline'

if not os.path.exists(REPO_DIR):
    !git clone --depth 1 {REPO_URL} {REPO_DIR}
else:
    %cd {REPO_DIR}
    !git pull

%cd {REPO_DIR}
!pip install -U pip
!pip install -r requirements.txt

## 3. Vérifier / ajuster les configurations

In [ ]:
!ls config
!cat config/generator.yml

## 4. Balayage métrique (facultatif mais conseillé)

In [ ]:
!python scripts/effect_metric_sweep.py \
        --config config/bins.yml \
        --output data/logs/effect_metric_sweep.csv
!head -n 5 data/logs/effect_metric_sweep.csv

## 5. Génération du dataset (multi-passes, couverture adaptative)

In [ ]:
DATASET_PATH = 'data/logs/train_dataset.jsonl'
!python scripts/generate_dataset.py \
        --config config/generator.yml \
        --num-workers 8 \
        --output {DATASET_PATH}
!tail -n 5 {DATASET_PATH}

## 6. Analyse rapide du dataset
Affiche les statistiques et génère des histogrammes (stockés dans `analysis/plots`).

In [ ]:
!python scripts/analyse_metrics.py \
        --input {DATASET_PATH} \
        --limit 5000 \
        --output-dir analysis/plots
!ls analysis/plots | head

## 7. Entraînement GNN

In [ ]:
LOG_DIR = 'data/logs/run_colab'
!python scripts/train_gnn.py \
        --config config/train.yml \
        --dataset {DATASET_PATH} \
        --log-dir {LOG_DIR} \
        --epochs 30 \
        --checkpoint-every 5

## 8. Visualiser les métriques d'entraînement

In [ ]:
import json
import matplotlib.pyplot as plt
with open(f'{LOG_DIR}/metrics_log.jsonl') as f:
    records = [json.loads(line) for line in f]
epochs = [r['epoch'] for r in records]
loss = [r['train_loss'] for r in records]
train_acc = [r['train_acc'] for r in records]
val_acc = [r['val_acc'] for r in records]
fig, ax1 = plt.subplots(figsize=(7,4))
ax1.plot(epochs, loss, color='tab:red', label='Loss')
ax1.set_xlabel('Epoch'); ax1.set_ylabel('Loss', color='tab:red')
ax2 = ax1.twinx()
ax2.plot(epochs, train_acc, color='tab:blue', label='Train acc')
ax2.plot(epochs, val_acc, color='tab:green', label='Val acc')
ax2.set_ylabel('Accuracy')
fig.legend(loc='upper center', bbox_to_anchor=(0.5,1.12), ncol=3)
plt.title('Suivi entraînement GNN')
plt.show()

## 9. Évaluation & inversion

In [ ]:
!python scripts/evaluate_gnn.py \
        --checkpoint {LOG_DIR}/ckpt_epoch30.pt \
        --dataset {DATASET_PATH} \
        --topk 5

## 10. Export des artifacts

In [ ]:
!tar -czf run_colab_artifacts.tar.gz {LOG_DIR} data/logs/effect_metric_sweep.csv
from google.colab import files
files.download('run_colab_artifacts.tar.gz')

## 11. Étapes suivantes
- Ajuster `config/generator.yml` pour des passes additionnelles / nouvelles familles.
- Étendre `config/bins.yml` si nécessaire.
- Lancer des entraînements distribués sur cluster GH200/H100.
- Intégrer le pipeline d'inversion et l'estimateur WebP dans un service production.